In [ ]:
# --- BƯỚC 1: KẾT NỐI DRIVE & CÀI ĐẶT MÔI TRƯỜNG ---
from google.colab import drive
import os

# 1. Mount Google Drive
if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

# 2. Kiểm tra GPU
print("✅ Kiểm tra GPU:")
!nvidia-smi

# 3. Cài đặt Darknet Framework
%cd /content/
if os.path.exists('darknet'):
    !rm -rf darknet

print("⬇️ Đang tải và biên dịch Darknet...")
!git clone https://github.com/AlexeyAB/darknet
%cd darknet

# Sửa Makefile để bật GPU và OpenCV
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

# Biên dịch
!make
print("✅ Cài đặt môi trường hoàn tất!")

In [ ]:
# --- BƯỚC 2: CHUẨN BỊ DỮ LIỆU & CẤU HÌNH ---
import os
import shutil
import glob

# 1. Copy và Giải nén dữ liệu từ Drive
drive_path = '/content/drive/MyDrive/obj.zip' # Đường dẫn file zip của bạn
if os.path.exists(drive_path):
    print("📦 Đang giải nén dữ liệu...")
    shutil.copy(drive_path, 'data/obj.zip')
    if os.path.exists('data/obj'): shutil.rmtree('data/obj')
    os.makedirs('data/obj')
    !unzip -q data/obj.zip -d data/obj/
else:
    print("⚠️ Lưu ý: Không tìm thấy file obj.zip trên Drive. Đảm bảo bạn đã upload file này.")

# 2. Tạo file danh sách ảnh (train.txt)
images_list = glob.glob("data/obj/**/*.jpg", recursive=True)
with open("data/train.txt", "w") as f:
    f.write("\n".join(images_list))
# Dùng chung tập train cho test (Demo)
shutil.copy("data/train.txt", "data/test.txt")

# 3. Tạo file định nghĩa lớp (obj.names)
with open("data/obj.names", "w") as f:
    f.write("license_plate")

# 4. Tạo file cấu hình đường dẫn (obj.data)
backup_dir = '/content/drive/MyDrive/DoAn_BienSoXe_Weights'
if not os.path.exists(backup_dir): os.makedirs(backup_dir)

with open("data/obj.data", "w") as f:
    f.write(f"classes= 1\n")
    f.write(f"train  = {os.getcwd()}/data/train.txt\n")
    f.write(f"valid  = {os.getcwd()}/data/test.txt\n")
    f.write(f"names  = {os.getcwd()}/data/obj.names\n")
    f.write(f"backup = {backup_dir}/\n")

# 5. Tạo file cấu hình mạng (yolov4-tiny-obj.cfg)
print("🛠️ Đang tạo file cấu hình mạng...")
!wget -q -O cfg/yolov4-tiny-custom.cfg https://raw.githubusercontent.com/AlexeyAB/darknet/master/cfg/yolov4-tiny.cfg

with open('cfg/yolov4-tiny-custom.cfg', 'r') as f: lines = f.readlines()
with open('cfg/yolov4-tiny-obj.cfg', 'w') as f:
    for line in lines:
        if line.startswith('batch='): f.write('batch=64\n')
        elif line.startswith('subdivisions='): f.write('subdivisions=16\n')
        elif line.startswith('max_batches='): f.write('max_batches=6000\n')
        elif line.startswith('steps='): f.write('steps=4800,5400\n')
        elif line.startswith('classes='): f.write('classes=1\n')
        elif line.startswith('filters=255'): f.write('filters=18\n')
        else: f.write(line)

print("✅ Đã chuẩn bị xong dữ liệu và cấu hình!")

In [ ]:
# --- BƯỚC 3: TIẾN HÀNH HUẤN LUYỆN ---
import os

# 1. Tải trọng số Pre-trained (Transfer Learning)
if not os.path.exists('yolov4-tiny.conv.29'):
    print("⬇️ Đang tải trọng số mẫu...")
    !wget -q https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29

# 2. Lệnh Huấn luyện
print("🚀 Bắt đầu huấn luyện mô hình...")
# Lưu ý: Lệnh này sẽ chạy lâu. Để xem log chi tiết, bỏ phần '> train.log'
!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -dont_show -map